In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pandas as pd

from sklearn.cluster import KMeans
from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

from utils import label_map_util
from utils import visualization_utils as vis_util

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

%matplotlib inline

/Users/oly/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/oly/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/oly/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/oly/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
model_name = 'inference_graph'
path_to_frozen_graph = model_name + '/frozen_inference_graph.pb'
path_to_labels = 'training/labelmap.pbtxt'

In [3]:
#create card values
suits = ['S','C','H','D']
card_vals = ['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']
n_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10]

hard_hands_index = np.arange(19)
hard_hands_values = np.arange(3,22)
    
hard_hand_vals_dict_reverse = {}
for i in range(len(hard_hands_index)): 
    hard_hand_vals_dict_reverse[hard_hands_values[i]] = hard_hands_index[i]
    
soft_hands_index = np.arange(19)
soft_hands_values = np.arange(3,22)
    
soft_hand_vals_dict_reverse = {}
for i in range(len(soft_hands_index)): 
    soft_hand_vals_dict_reverse[soft_hands_values[i]] = soft_hands_index[i]
    
dealer_hands_index = np.arange(10)
dealer_hands_values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    
dealer_hand_vals_dict_reverse = {}
for i in range(len(dealer_hands_index)): 
    dealer_hand_vals_dict_reverse[dealer_hands_values[i]] = dealer_hands_index[i]

    
#Load in the decision matrix
hard_matrix = np.loadtxt('Blackjack_data/dm_hard_hand.txt').reshape(len(hard_hands_index), len(dealer_hands_index))
soft_matrix = np.loadtxt('Blackjack_data/dm_soft_hand.txt').reshape(len(soft_hands_index), len(dealer_hands_index))

In [4]:
card_values = pd.read_csv('Blackjack_data/card_values.csv')
card_values = np.array(card_values)
card_vals_dict = {}
for i in range(len(card_values)):
    card_vals_dict[card_values[i][0]] = card_values[i][1]
    
    
ace_dict = {}
for i in card_vals_dict:
    if i[0] == 'A':
        ace = 1
    else: ace = 0
    ace_dict[i] = ace

In [5]:
def get_hand_type(hand, dealer_flag, card_vals_dict, ace_dict):
    val = 0
    aces = 0
    vals = []
    has_pair = 0
    
    for i in hand:
        val += card_vals_dict[i]
        aces += ace_dict[i]
        vals.append(i[0])
        
    if len(np.unique(vals)) < len(vals):
        has_pair = 1
    
    #if the ace there is an ace then use the sum of the aces to increase the value
    if aces > 0:
        if val + (10*aces) <= 21:
            val+= 10
            
    if aces > 0:
        hand_type = 1 #soft
    else:
        if has_pair == 1:
            hand_type = 2 #pair
        else: hand_type = 0 #hard
        
    if dealer_flag == 1:
        return(val)
    else:   
        return(val, hand_type)

In [8]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(path_to_frozen_graph, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [9]:
category_index = label_map_util.create_category_index_from_labelmap(path_to_labels, use_display_name=True)

In [10]:
class_label_dict = {}
for i in category_index:
    class_label_dict[category_index[i]['id']] = category_index[i]['name']

In [11]:
def run_inference_for_single_image(image, graph):
    if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

    # Run inference
    output_dict = sess.run(tensor_dict,
                            feed_dict={image_tensor: np.expand_dims(image, 0)})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [14]:
import cv2
import requests


past_image = []
message = 'new game'

url = 'http://192.168.0.37:8080/shot.jpg'

#cap = cv2.VideoCapture(0)

with detection_graph.as_default():
    with tf.Session() as sess:
        # Get handles to input and output tensors
        ops = tf.get_default_graph().get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in [
         'num_detections', 'detection_boxes', 'detection_scores',
         'detection_classes', 'detection_masks']:
            tensor_name = key + ':0'
            if tensor_name in all_tensor_names:
                tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)

        while True:

            img_resp = requests.get(url)
            img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
            img = cv2.imdecode(img_arr, -1)

            image_np = img
            if len(past_image) == 0:
                past_image = image_np.copy()


            #Check if the image is the same as the previous image
            #If it is, stop here. 
            #If it's not the same, continue

            if (image_np == past_image).all():
                if message == 'Do Nothing same image':
                    z = 0
                else: 
                    message = 'Do Nothing same image'
                    print(message)
            else:

                output_dict = run_inference_for_single_image(image_np, detection_graph)

                # Visualization of the results of a detection.
                vis_util.visualize_boxes_and_labels_on_image_array(
                   image_np,
                   output_dict['detection_boxes'],
                   output_dict['detection_classes'],
                   output_dict['detection_scores'],
                   category_index,

                   instance_masks=output_dict.get('detection_masks'),
                   use_normalized_coordinates=True,
                   line_thickness=7) #was 8
                
                finder = output_dict['detection_boxes'][output_dict['detection_scores'] > 0.7] #Get co-ordinates of detected bounding boxes
                detected_classes = output_dict['detection_classes'][output_dict['detection_scores'] > 0.7] #Get the detected classes of the cards

                #check how many cards detected
                #If the length of finder == 0 then this means no cards in the playing area:
                #set a variable to say new hand

                if len(finder) < 3:
                    new_hand = 1
                    if message == 'New Hand - Waiting For Cards to be Dealt':
                        z = 0
                    else: 
                        message = 'New Hand - Waiting For Cards to be Dealt'
                        print(message)
                else: new_hand = 0

                if new_hand == 0:

                    #if the length of finder is == 3 this means 3 cards are detected
                    #define a flag to say starting hand
                    #Train the kmeans on the placement

                    if len(finder) == 3:
                        Kmeans = KMeans(n_clusters=2).fit(np.array(finder)) #Train Kmeans based on the initial hand

                        player_a = 0

                        for i in range(len(finder)):
                            if Kmeans.predict(finder[i].reshape(1,-1)) == 0:
                                player_a += 1

                        if player_a > 1:
                            player = 0
                            dealer = 1
                        else: 
                            player = 1
                            dealer = 0  


                        hand_location = Kmeans.predict(finder)

                        player_hand = hand_location[hand_location == player]
                        dealer_hand = hand_location[hand_location == dealer]
                        
                        player_hand_class = detected_classes[hand_location == player]
                        dealer_hand_class = detected_classes[hand_location == dealer]

                        player_hand_class = [class_label_dict[x] for x in player_hand_class]
                        dealer_hand_class = [class_label_dict[x] for x in dealer_hand_class]
                        
                        player_hand_value = 0
                        dealer_hand_value = 0
                        player_ace = 0
                        dealer_ace = 0

                        for i in player_hand_class:
                            player_hand_value += card_vals_dict[str(i)]
                            player_ace += ace_dict[str(i)]

                        if player_ace > 0:
                            if player_hand_value + 10 < 22:
                                player_hand_value += 10

                        for i in dealer_hand_class:
                            dealer_hand_value += card_vals_dict[str(i)]
                            dealer_ace += ace_dict[str(i)]

                        if dealer_ace > 0:
                            dealer_hand_value += 10

                    #define from the kmeans which hand belongs to who and set this

                    #if the length of finder > 3:
                    #the hand is continguing
                    #pull in the variable for each players kmeans hand number
                    #assign the hand values to each player

                    if len(finder) > 3:
                        hand_location = Kmeans.predict(finder)

                        player_hand = hand_location[hand_location == player]
                        dealer_hand = hand_location[hand_location == dealer]
                        
                        player_hand_class = detected_classes[hand_location == player]
                        dealer_hand_class = detected_classes[hand_location == dealer]

                        player_hand_class = [class_label_dict[x] for x in player_hand_class]
                        dealer_hand_class = [class_label_dict[x] for x in dealer_hand_class]
                        
                        player_hand_value = 0
                        dealer_hand_value = 0
                        player_ace = 0
                        dealer_ace = 0

                        for i in player_hand_class:
                            player_hand_value += card_vals_dict[str(i)]
                            player_ace += ace_dict[str(i)]

                        if player_ace > 0:
                            if player_hand_value + 10 < 22:
                                player_hand_value += 10

                        for i in dealer_hand_class:
                            dealer_hand_value += card_vals_dict[str(i)]
                            dealer_ace += ace_dict[str(i)]

                        if dealer_ace > 0:
                            dealer_hand_value += 10

                    #If the dealer is only holding 1 card                
                    #cross reference to the decision matrix to get the answer of what to do
                    #print the answer of what to do

                    if len(dealer_hand) < 2:
                        #get what to do from the player hand
                        #print('dealer only has 1 card')
                        
                        if player_hand_value == 21:
                            if message == 'Dealer has'+str(dealer_hand_value)+'I have'+str(player_hand_value)+'Blackjack! I Win!!':
                                z = 0
                            else:
                                message = 'Dealer has'+str(dealer_hand_value)+'I have'+str(player_hand_value)+'Blackjack! I Win!!'
                                print('Dealer has', dealer_hand_value)
                                print('I have', player_hand_value)
                                print('Blackjack! I win!!')
                                
                        else:
                            if player_hand_value > 21:
                                if message == 'Dealer has'+str(dealer_hand_value)+'I Have'+str(player_hand_value)+'I Lose!':
                                    z = 0
                                else: 
                                    message = 'Dealer has'+str(dealer_hand_value)+'I Have'+str(player_hand_value)+'I Lose!'
                                    print('Dealer has', dealer_hand_value)
                                    print('I Have', player_hand_value)
                                    print('I Lose!')
                            else:                                
                                
                                if player_ace == 0:
                                    decision = hard_matrix[hard_hand_vals_dict_reverse[player_hand_value]][hard_hand_vals_dict_reverse[dealer_hand_value]]
                                else: soft_matrix[soft_hand_vals_dict_reverse[player_hand_value]][soft_hand_vals_dict_reverse[dealer_hand_value]]
                                    
                                if decision == 1:
                                    choice_str = 'I will have a card please'
                                else: choice_str = 'I will stick, no card for me'

                                if message == 'I have'+str(player_hand_value)+'The dealer has'+str(dealer_hand_value)+choice_str:
                                    z = 0
                                else:
                                    message = 'I have'+str(player_hand_value)+'The dealer has'+str(dealer_hand_value)+choice_str
                                    print('I have', player_hand_value)
                                    print('The dealer has', dealer_hand_value)
                                    print(choice_str)
                    else:
                        if dealer_hand_value < 17:
                            if message == 'Dealer has'+str(dealer_hand_value)+'I have'+str(player_hand_value)+'Dealer less than 17, must turn again':
                                z = 0
                            else: 
                                message = 'Dealer has'+str(dealer_hand_value)+'I have'+str(player_hand_value)+'Dealer less than 17, must turn again'         
                                print('Dealer has', dealer_hand_value)
                                print('I have', player_hand_value)
                                print('Dealer less than 17, must turn again')
                        else:
                            if dealer_hand_value > 21:
                                if message == 'Dealer has'+str(dealer_hand_value)+'I have'+str(player_hand_value)+'Dealer is bust, I win!!':
                                    z = 0
                                else:
                                    message = 'Dealer has'+str(dealer_hand_value)+'I have'+str(player_hand_value)+'Dealer is bust, I win!!'
                                    print('Dealer has', dealer_hand_value)
                                    print('I have', player_hand_value)
                                    print('Dealer is bust, I win!!')
                            else:
                                if player_hand_value > dealer_hand_value:
                                    if message == 'Dealer has'+str(dealer_hand_value)+'I have'+str(player_hand_value)+'I win!':
                                        z = 0
                                    else:
                                        message = 'Dealer has'+str(dealer_hand_value)+'I have'+str(player_hand_value)+'I win!'
                                        print('Dealer has', dealer_hand_value)
                                        print('I have', player_hand_value)
                                        print('I win!')
                                else:
                                    if player_hand_value == dealer_hand_value:
                                        if message == 'Dealer has'+str(dealer_hand_value)+'I have'+str(player_hand_value)+'We draw!':
                                            z = 0
                                        else:
                                            message = 'Dealer has'+str(dealer_hand_value)+'I have'+str(player_hand_value)+'We draw!'
                                            print('Dealer has', dealer_hand_value)
                                            print('I have', player_hand_value)
                                            print('We draw!')
                                    else:
                                        if player_hand_value < dealer_hand_value:
                                            if message == 'Dealer has'+str(dealer_hand_value)+'I have'+str(player_hand_value)+'Dealer Wins, I loose!':
                                                z = 0
                                            else: 
                                                message = 'Dealer has'+str(dealer_hand_value)+'I have'+str(player_hand_value)+'Dealer Wins, I loose!'
                                                print('Dealer has', dealer_hand_value)
                                                print('I have', player_hand_value)
                                                print('Dealer Wins, I loose!')


            past_image = image_np.copy()                  

            cv2.imshow('object_detection', cv2.resize(image_np, (700, 500)))
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break

Do Nothing same image
I have 13
The dealer has 7
I will stick, no card for me
Dealer has 17
I have 13
Dealer Wins, I loose!
I have 13
The dealer has 9
I will stick, no card for me
New Hand - Waiting For Cards to be Dealt
I have 12
The dealer has 10
I will have a card please
New Hand - Waiting For Cards to be Dealt
Dealer has 10
I Have 22
I Lose!
I have 12
The dealer has 10
I will have a card please
New Hand - Waiting For Cards to be Dealt
I have 14
The dealer has 10
I will have a card please
I have 14
The dealer has 9
I will have a card please
I have 20
The dealer has 9
I will have a card please
I have 19
The dealer has 9
I will have a card please
I have 17
The dealer has 9
I will have a card please
Dealer has 9
I Have 25
I Lose!
I have 9
The dealer has 11
I will have a card please
New Hand - Waiting For Cards to be Dealt
I have 13
The dealer has 9
I will stick, no card for me
I have 15
The dealer has 10
I will have a card please
Dealer has 15
I have 15
Dealer less than 17, must turn a

KeyboardInterrupt: 